**En este proyecto se aplicará Knn en la predicción de emojis**


Primero, importaré las librerías necesarias junto con el dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from gensim.models import Word2Vec
import gensim.downloader as api
from sklearn.feature_extraction.text import TfidfVectorizer
from google.colab import drive
import seaborn as sns

drive.mount('/content/drive')

ruta_archivo = '/content/drive/MyDrive/dataset_emojis.csv'
df = pd.read_csv(ruta_archivo, encoding='utf-8')
print(df)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   Categoría                       Frase
0          😊             Me alegra mucho
1          😊                 Estoy feliz
2          😂          Que divertido eres
3          😂                Es divertido
4          🤣           Eres muy chistoso
5          🤣                  ¡Qué risa!
6         ❤️                   Te quiero
7          😍               Eres mi crush
8          😍                   Me gustas
9          😎                    Soy cool
10         😘              Te doy un beso
11         😘            Te mando un beso
12         🤑             Ya cayó la beca
13         🤑                  Ya pagaron
14         🫡                  A la orden
15         🫡                  Entendido.
16         🤔                    No lo sé
17         🤔               Esoy pensando
18         👌                 ¡Excelente!
19         👌                  ¡Perfecto!
20        

Lo siguiente es hacer la representación vectorial para las palabras usando Word2Vec, pero esta parte permite elegir entre utilizar Word2Vec o TF-IDF.

In [2]:
# Preprocesamiento básico (tokenización)
df['tokens'] = df['Frase'].apply(lambda x: x.split())

# Permite elegir la representación vectorial
vectorization_method = "word2vec"

if vectorization_method == "word2vec":
    try:
        word2vec_model = api.load("word2vec-google-news-300")
        print("Modelo Word2Vec cargado")
    except ValueError:
        print("Modelo no encontrado, descargando...")
        word2vec_model = api.load("word2vec-google-news-300")
        print("Modelo Word2Vec descargado")

    def obtener_embedding_word2vec(tokens):
        embeddings = [word2vec_model[token] for token in tokens if token in word2vec_model]
        return np.mean(embeddings, axis=0) if embeddings else np.zeros(word2vec_model.vector_size)

    df['embedding'] = df['tokens'].apply(obtener_embedding_word2vec)
    X = np.array(list(df['embedding']))

elif vectorization_method == "tfidf":
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(df['Frase']).toarray()
    print("Usando TF-IDF")

Modelo Word2Vec cargado


Ahora, la codificación de los emojis para poder trabajar con ellos.

In [3]:
# Codificación de emojis
df['emojis_lista'] = df['Categoría'].apply(lambda x: [x] if isinstance(x, str) else [])
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['emojis_lista'])
y_labels_text = [str(label) for label in mlb.classes_]

Lo siguiente es dividir los datos de entrenamiento y de prueba, y entrenaré el modelo con búsqueda del mejor k usando validación cruzada.

In [4]:
# Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Búsqueda del mejor k usando validación cruzada
best_k = 1
best_score = 0
for k in range(1, 11):  # Prueba valores de k del 1 al 10
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train, y_train, cv=5)
    if np.mean(scores) > best_score:
        best_score = np.mean(scores)
        best_k = k

# Entrenar el modelo KNN
print(f"Mejor valor de k encontrado: {best_k}")
knn = KNeighborsClassifier(n_neighbors=best_k)
knn.fit(X_train, y_train)

Mejor valor de k encontrado: 1


KNeighborsClassifier(n_neighbors=1)

Ahora, haré la evolución y presentaré predicciones con ejemplos de predicciones correctas e incorrectas

In [8]:
# Predicciones y Evaluación
y_pred = knn.predict(X_test)

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=y_labels_text, zero_division=0))
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")


# Análisis de predicciones correctas e incorrectas (ejemplo)
incorrect_predictions = np.where(np.any(y_pred != y_test, axis=1))[0]
correct_predictions = np.where(np.all(y_pred == y_test, axis=1))[0]


print("\nEjemplos de predicciones INCORRECTAS:")
for i in incorrect_predictions[:5]:
    print(f"Frase: {df['Frase'].iloc[i]}")
    print(f"Emoji Real: {mlb.inverse_transform(y_test[i].reshape(1, -1))}")
    print(f"Emoji Predicho: {mlb.inverse_transform(y_pred[i].reshape(1, -1))}\n")

print("\nEjemplos de predicciones CORRECTAS:")
for i in correct_predictions[:5]:
    print(f"Frase: {df['Frase'].iloc[i]}")
    print(f"Emoji Real: {mlb.inverse_transform(y_test[i].reshape(1, -1))}")
    print(f"Emoji Predicho: {mlb.inverse_transform(y_pred[i].reshape(1, -1))}\n")


Classification Report:
              precision    recall  f1-score   support

          ❤️       0.00      0.00      0.00         1
           👌       0.00      0.00      0.00         0
           😂       0.00      0.00      0.00         0
           😊       0.00      0.00      0.00         0
           😍       0.00      0.00      0.00         0
           😎       0.00      0.00      0.00         0
           😘       0.00      0.00      0.00         0
           😩       0.00      0.00      0.00         0
           😭       0.00      0.00      0.00         2
           😷       0.00      0.00      0.00         1
           🙌       0.00      0.00      0.00         0
           🤑       1.00      1.00      1.00         1
           🤔       0.00      0.00      0.00         0
           🤢       0.00      0.00      0.00         0
           🤣       0.00      0.00      0.00         1
           🤯       0.00      0.00      0.00         0
        🤷‍♂️       0.00      0.00      0.00         0
   

Los ejemplos de predicciones incorrectas son, efectivamente, incorrectos. Por otra parte, los ejemplos correctos si muestran el emoji correcto, aunque tal vez la frase parezca no ser la más acertada.